# Set-up Environment

In [1]:
spark.version

'3.1.3'

In [2]:
import os
import pandas as pd
import numpy as np

from itertools import compress 
from pyspark.sql.functions import *
from pyspark.sql.types import *
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
warnings.filterwarnings(action = 'ignore')

In [3]:
!pip3 install nltk -U

In [4]:
import re
import json
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.feature import CountVectorizer, IDF, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from google.cloud import storage

spark.conf.set("spark.sql.repl.eagerEval.enabled",True)
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

## Set-up COS functions for GCP

In [6]:
# List all files in given COS directory
def list_blobs_pd(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix = folder_name))

    blob_name = []
    blob_size = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size)), columns = ['Name','Size'])

    #blobs_df = blobs_df.style.format({"Size": "{:,.0f}"}) 
    
    return blobs_df

In [7]:
# Delete folder from COS bucket
def delete_folder(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix = folder_name))

    for blob in blobs:
        blob.delete()

In [8]:
# Reading data from bucket
bucket = 'shijia-bdp-class'

# Read Twitter Data JSON Files

In [9]:
blob_list = list_blobs_pd(bucket, 'final_project/')

In [10]:
blob_list.head(10)

,Name,Size
0,final_project/,0
1,final_project/_SUCCESS,0
2,final_project/part-00000-f654a635-796b-4190-88ae-6c2ee7e6f3a3-c000.json,6007332
3,final_project/part-00001-f654a635-796b-4190-88ae-6c2ee7e6f3a3-c000.json,5638649
4,final_project/part-00002-f654a635-796b-4190-88ae-6c2ee7e6f3a3-c000.json,6649652
5,final_project/part-00003-f654a635-796b-4190-88ae-6c2ee7e6f3a3-c000.json,6921001
6,final_project/part-00004-f654a635-796b-4190-88ae-6c2ee7e6f3a3-c000.json,6626757
7,final_project/part-00005-f654a635-796b-4190-88ae-6c2ee7e6f3a3-c000.json,6696245
8,final_project/part-00006-f654a635-796b-4190-88ae-6c2ee7e6f3a3-c000.json,6658398
9,final_project/part-00007-f654a635-796b-4190-88ae-6c2ee7e6f3a3-c000.json,4963067


## Explore the data structure in a small sample

In [11]:
# read into spark dataframe (a small subgroup as sample)
path = 'gs://shijia-bdp-class/final_project/part-2468*'
tweets_raw_sample = spark.read.json(path)

In [12]:
tweets_raw_sample.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

In [13]:
tweets_raw_sample.count()

36949

In [14]:
tweets_raw_sample.limit(20)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_text,reply_count,retweet_count,retweeted,retweeted_from,retweeted_status,source,text,timestamp_ms,truncated,tweet_text,user,withheld_in_countries
null,Fri Jun 03 19:48:...,null,"{[], null, [], []...",null,null,0,false,low,null,1532811539777994753,1532811539777994753,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,BingoDemagogue,"{null, Fri Jun 03...","<a href=""http://t...",RT @BingoDemagogu...,1654285724001,false,I hope you've fou...,"{false, Fri Nov 0...",null
null,Fri Jun 03 19:48:...,"[16, 140]","{[], null, [], [{...",null,"{[16, 173], {[], ...",0,false,low,null,1532811540507901952,1532811540507901952,rising_serpent,1532778289202348033,1532778289202348033,879548970082721792,879548970082721792,false,en,null,null,0,null,null,null,null,null,0,0,,null,null,"<a href=""https://...",@rising_serpent I...,1654285724175,true,@rising_serpent I...,"{false, Sun Oct 1...",null
null,Fri Jun 03 19:48:...,null,"{[], null, [], []...",null,null,0,false,low,null,1532811540813987843,1532811540813987843,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,donwinslow,"{null, Fri Jun 03...","<a href=""http://t...",RT @donwinslow: D...,1654285724248,false,Dear @RepKinzinge...,"{false, Fri Oct 0...",null
null,Fri Jun 03 19:48:...,null,"{[], null, [], []...",null,null,0,false,low,null,1532811540922916864,1532811540922916864,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,PatMcAfeeShow,"{null, Fri Jun 03...","<a href=""http://t...",RT @PatMcAfeeShow...,1654285724274,false,He’s a College Fo...,"{false, Mon Jun 1...",null
null,Fri Jun 03 19:48:...,"[0, 140]","{[], null, [], [{...",null,"{[0, 276], {[], [...",0,false,low,null,1532811540575014912,1532811540575014912,null,null,null,null,null,false,en,null,false,0,null,null,null,null,null,0,0,,null,null,"<a href=""https://...",Happy pride month...,1654285724191,true,Happy pride month...,"{false, Sat Dec 1...",null
null,Fri Jun 03 19:48:...,null,"{[], null, [], []...",null,null,0,false,low,null,1532811541753503748,1532811541753503748,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,shortney0006,"{null, Wed Jun 01...","<a href=""http://t...",RT @shortney0006:...,1654285724472,false,Today’s shooting ...,"{false, Fri Nov 3...",null
null,Fri Jun 03 19:48:...,null,"{[], null, [], []...",null,null,0,false,low,null,1532811542709796864,1532811542709796864,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,POTUS,"{null, Thu Jun 02...","<a href=""http://t...",RT @POTUS: Over t...,1654285724700,false,Over the last two...,"{false, Fri Dec 2...",null
null,Fri Jun 03 19:48:...,null,"{[{[87, 93], horn...",null,null,0,false,low,null,1532811543741714436,1532811543741714436,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,Brisa_ox,"{null, Fri Jun 03...","<a href=""http://t...",RT @Brisa_ox: Get...,1654285724946,false,Getting ready for...,"{false, Tue Mar 2...",null
null,Fri Jun 03 19:48:...,null,"{[], null, [], []...",null,null,0,false,low,null,1532811543972290561,1532811543972290561,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,ZekeJMiller,"{null, Fri Jun 03...","<a href=""http://t...",RT @ZekeJMiller: ...,1654285725001,false,"UVALDE, Texas (AP...","{false, Wed Aug 1...",null
null,Fri Jun 03 19:48:...,null,"{[], null, [], []...",null,null,0,false,low,null,1532811544253411330,1532811544253411330,null,null,null,null,null,false,en,null,null,0,null,null,null,null,null,0,0,RT,TechAthletics,"{null, Fri Jun 03...","<a href=""http://t...",RT @TechA

## Expand and explore the nested columns

### Coordinates

In [15]:
coordinates = tweets_raw_sample.select('coordinates.*')
coordinates.printSchema()

root
 |-- coordinates: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- type: string (nullable = true)



In [16]:
coordinates.limit(10)

coordinates,type
null,null
null,null
null,null
null,null
null,null
null,null
null,null
null,null
null,null
null,null


### User

In [17]:
user = tweets_raw_sample.select('user.*')
user.printSchema()

root
 |-- contributors_enabled: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- default_profile: boolean (nullable = true)
 |-- default_profile_image: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- geo_enabled: boolean (nullable = true)
 |-- id: long (nullable = true)
 |-- id_str: string (nullable = true)
 |-- is_translator: boolean (nullable = true)
 |-- listed_count: long (nullable = true)
 |-- location: string (nullable = true)
 |-- name: string (nullable = true)
 |-- profile_background_color: string (nullable = true)
 |-- profile_background_image_url: string (nullable = true)
 |-- profile_background_image_url_https: string (nullable = true)
 |-- profile_background_tile: boolean (nullable = true)
 |-- profile_banner_url: string (nullable = true)
 |-- profile_image_url: string (nullable = true)
 |-- p

In [18]:
user.limit(10)

contributors_enabled,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,id_str,is_translator,listed_count,location,name,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,translator_type,url,verified,withheld_in_countries
false,Thu Feb 24 02:59:...,true,false,°☆.:SATIRE ACCOUN...,748,80,857,false,1496681057206382595,1496681057206382595,false,0,mouse gender sqea...,iheartdietculture...,F5F8FA,,,false,https://pbs.twimg...,http://pbs.twimg....,https://pbs.twimg...,1DA1F2,C0DEED,DDEEF6,333333,true,false,Tradconspiracy6,2731,none,null,false,[]
false,Tue Dec 23 17:32:...,true,false,BRAINIAC MOM OF 7...,7998,41,162,true,2941041980,2941041980,false,1,null,Annette Di Re,C0DEED,http://abs.twimg....,https://abs.twimg...,false,null,http://pbs.twimg....,https://pbs.twimg...,1DA1F2,C0DEED,DDEEF6,333333,true,false,annettedire14,4724,none,null,false,[]
false,Tue Nov 02 21:54:...,true,false,null,2342,152,261,true,1455654488086351878,1455654488086351878,false,0,"Manchester, England",EDL 🇹🇷,F5F8FA,,,false,https://pbs.twimg...,http://pbs.twimg....,https://pbs.twimg...,1DA1F2,C0DEED,DDEEF6,333333,true,false,theybysheree,1076,none,https://dialectic...,false,[]
false,Sat Aug 31 09:35:...,true,false,I AM A MAVIN @D...,101352,1697,712,false,1715194136,1715194136,false,10,"Osapa, London",Baroness of the Q...,C0DEED,http://abs.twimg....,https://abs.twimg...,false,https://pbs.twimg...,http://pbs.twimg....,https://pbs.twimg...,1DA1F2,C0DEED,DDEEF6,333333,true,false,thelastceleb,42223,none,https://paystack....,false,[]
false,Tue Jan 11 01:26:...,true,false,#Independent #Bre...,54334,390,731,false,1480712553156300804,1480712553156300804,false,24,"Virginia, USA",💫Mary Joanne🤍💫,F5F8FA,,,false,https://pbs.twimg...,http://pbs.twimg....,https://pbs.twimg...,1DA1F2,C0DEED,DDEEF6,333333,true,false,MaryJoanne2022,65093,none,null,false,[]
false,Thu Nov 18 13:27:...,true,false,null,5470,36,410,false,1461325181453049858,1461325181453049858,false,2,null,mikeinsta06450@ya...,F5F8FA,,,false,null,http://abs.twimg....,https://abs.twimg...,1DA1F2,C0DEED,DDEEF6,333333,true,false,mikeinsta06450,7648,none,null,false,[]
false,Mon Jan 22 07:46:...,false,false,"Family, Sports, e...",15276,431,751,true,955345926087901186,955345926087901186,false,1,South Chicago Hei...,That Celtics fan.,000000,http://abs.twimg....,https://abs.twimg...,false,https://pbs.twimg...,http://pbs.twimg....,https://pbs.twimg...,FF691F,000000,000000,000000,false,false,asiimwefelix22,19191,none,https://asiimwero...,false,[]
false,Sat Mar 20 19:37:...,true,false,M◐◑N palace - 01/...,23176,235,248,true,1373357975709048837,1373357975709048837,false,0,he / they / xe 1...,Nyx⁸ ⩜⃝,F5F8FA,,,false,https://pbs.twimg...,http://pbs.twimg....,https://pbs.twimg...,1DA1F2,C0DEED,DDEEF6,333333,true,false,Iapisena,20613,none,null,false,[]
false,Sat May 29 22:03:...,true,false,null,27,3,19,false,1398761900213784577,1398761900213784577,false,0,null,Mattus,F5F8FA,,,false,https://pbs.twimg...,http://pbs.twimg....,https://pbs.twimg...,1DA1F2,C0DEED,DDEEF6,333333,true,false,Mattus_X,737,none,null,false,[]
false,Sun Oct 03 15:18:...,true,false,The official twit...,40,76,118,true,1444683311910096902,1444683311910096902,false,1,"Walker, LA",Walker Esports Wi...,F5F8FA,,,false,https://pbs.twimg...,http://pbs.twimg....,https://pbs.twimg...,1DA1F2,C0DEED,DDEEF6,333333,true,false,WalkerEsports_,213,none,http://www.instag...,false,[]


### Entities

In [19]:
entities = tweets_raw_sample.select('entities.*')
entities.printSchema()

root
 |-- hashtags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- text: string (nullable = true)
 |-- media: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- display_url: string (nullable = true)
 |    |    |-- expanded_url: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- id_str: string (nullable = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- media_url: stri

In [20]:
entities.limit(10)

hashtags,media,symbols,urls,user_mentions
[],null,[],[],"[{128301595, 1283..."
[],null,[],[],[{154776378655926...
[],null,[],[],[]
[],null,[],[],[{857222466942500...
[],null,[],[{zerohedge.com/p...,"[{18856867, 18856..."
[],null,[],[],[{135008190312721...
[],null,[],[],"[{15012486, 15012..."
[],null,[],[],[]
[],null,[],[],"[{4784988439, 478..."
[],null,[],[],[{117322377934820...


In [21]:
hashtags = entities.select('hashtags.text')
hashtags.printSchema()

root
 |-- text: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [22]:
hashtags.limit(10)

text
[]
[]
[]
[]
"[mlbtwt, MLBS5Spo..."
[]
[]
[]
[ChandigarhUniver...
[]


### Extended Entities

In [23]:
extended_entities = tweets_raw_sample.select('extended_entities.*')
extended_entities.printSchema()

root
 |-- media: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- display_url: string (nullable = true)
 |    |    |-- expanded_url: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- id_str: string (nullable = true)
 |    |    |-- indices: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- media_url: string (nullable = true)
 |    |    |-- media_url_https: string (nullable = true)
 |    |    |-- sizes: struct (nullable = true)
 |    |    |    |-- large: struct (nullable = true)
 |    |    |    |    |-- h: long (nullable = true)
 |    |  

In [24]:
extended_entities.limit(10)

media
null
null
null
null
null
null
null
null
null
null


### Extended Tweet

In [25]:
extended_tweet = tweets_raw_sample.select('extended_tweet.*')
extended_tweet.printSchema()

root
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |    |-- title: string (nullable = true)
 |    |    |    |-- description: string (nullable = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |  

In [26]:
extended_tweet.limit(20)

display_text_range,entities,extended_entities,full_text
null,null,null,null
"[85, 364]","{[], null, [], []...",null,@LGBrandonPOS @Sh...
"[0, 205]","{[{[137, 142], ny...",null,Join OutLaws and ...
null,null,null,null
null,null,null,null
null,null,null,null
null,null,null,null
"[54, 235]","{[], null, [], []...",null,@andrewlbyrne @1b...
"[0, 277]","{[{[76, 102], Cha...","{[{null, null, pi...",@ugc_india @EduMi...
"[17, 293]","{[], null, [], []...",null,@abhijitmajumder ...


### Quoted Status

In [27]:
quoted_status = tweets_raw_sample.select('quoted_status.*')
quoted_status.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

In [28]:
quoted_status.limit(10)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status_id,quoted_status_id_str,reply_count,retweet_count,retweeted,scopes,source,text,truncated,user,withheld_in_countries
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


### Retweeted Status

In [29]:
retweeted_status = tweets_raw_sample.select('retweeted_status.*')
retweeted_status.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

In [30]:
retweeted_status.limit(10)

coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,scopes,source,text,truncated,user,withheld_in_countries
null,Fri May 20 13:40:...,null,"{[], null, [], [{...",null,"{[0, 150], {[], n...",82660,false,low,null,1527645516258955266,1527645516258955266,null,null,null,null,null,false,en,null,null,1624,null,null,null,null,5989,8760,false,null,"<a href=""http://t...",Super excited to ...,true,"{false, Tue Jun 0...",null
null,Fri May 20 05:34:...,"[0, 140]","{[{[0, 5], SUHO},...",null,"{[0, 276], {[{[0,...",48620,false,low,null,1527523090611068929,1527523090611068929,null,null,null,null,null,false,en,null,false,2858,null,null,null,null,398,22314,false,null,"<a href=""http://t...",#SUHO of #EXO att...,true,"{false, Tue Oct 1...",null
null,Wed May 18 20:15:...,null,"{[], null, [], [{...",null,"{[0, 277], {[], n...",35194,false,low,null,1527020147462049796,1527020147462049796,null,null,null,null,null,false,en,null,null,606,null,null,null,null,1781,15209,false,null,"<a href=""https://...",BREAKING: Arizona...,true,"{false, Sat Oct 1...",null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
null,Fri May 20 12:23:...,"[0, 140]","{[], null, [], [{...",null,"{[0, 216], {[], [...",4170,false,low,null,1527626038372892672,1527626038372892672,null,null,null,null,null,false,en,null,false,110,null,null,null,null,21,2310,false,null,"<a href=""https://...",LOOK: Suho of wor...,true,"{false, Wed Jul 1...",null
null,Fri May 20 13:40:...,null,"{[], null, [], [{...",null,"{[0, 150], {[], n...",82476,false,low,null,1527645516258955266,1527645516258955266,null,null,null,null,null,false,en,null,null,1628,null,null,null,null,6005,8755,false,null,"<a href=""http://t...",Super excited to ...,true,"{false, Tue Jun 0...",null
null,Fri May 20 01:16:...,null,"{[], null, [], [{...",null,"{[0, 275], {[], n...",10259,false,low,null,1527458191054757888,1527458191054757888,null,null,null,null,null,false,en,null,null,357,null,null,null,null,967,2754,false,null,"<a href=""http://t...",THREAD: My Profes...,true,"{false, Sun May 0...",null
null,Fri May 20 13:40:...,null,"{[], null, [], [{...",null,"{[0, 150], {[], n...",82598,false,low,null,1527645516258955266,1527645516258955266,null,null,null,null,null,false,en,null,null,1542,null,null,null,null,5995,8755,false,null,"<a href=""http://t...",Super excited to ...,true,"{false, Tue Jun 0...",null
null,Thu May 19 14:26:...,null,"{[], null, [], [{...",null,"{[0, 271], {[{[25...",12366,false,low,null,1527294702235443200,1527294702235443200,null,null,null,null,null,false,en,null,null,198,null,null,null,null,37,6778,false,null,"<a href=""http://t...",An Afghan-Muslim ...,true,"{false, Sat May 2...",null


# Selecting Variables Related to Twitterers

#### Based on the exploration, the following main variables are selected that are related to the problem for further analysis.

In [ ]:
# Read the full dataset
path = 'gs://shijia-bdp-class/final_project'
tweets_raw = spark.read.json(path)

22/12/03 23:10:42 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [ ]:
fields = ('coordinates', 'created_at', 'id', 'text','user', 'quote_count', 
          'reply_count', 'favorite_count','retweet_count', 'retweeted', 'retweeted_from', 
          'retweeted_status')

tweets = tweets_raw.select(*fields)

tweets.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- friends_count: long (nullable = true)
 |    |-- geo_enabled: boolean (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- id_str: string (nullable = true)
 |    |-- is_translator: boolean (nullable = true)
 |    |-- listed_count: long (nullable = true)
 |    |-- location: stri

## Select relevent fields in nested columns

#### Drop nested fields which contain large number of uncessary information in 'retweeted_status'

In [ ]:
all_fields_retweet = tweets.select("retweeted_status.*").columns
all_fields_retweet

['coordinates',
 'created_at',
 'display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorite_count',
 'favorited',
 'filter_level',
 'geo',
 'id',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quote_count',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'reply_count',
 'retweet_count',
 'retweeted',
 'scopes',
 'source',
 'text',
 'truncated',
 'user',
 'withheld_copyright',
 'withheld_in_countries']

In [ ]:
select_fields_retweet = ('coordinates', 'created_at', 'id', 'text', 'user', 'quote_count', 'reply_count', 'favorite_count', 'retweet_count', 'retweeted')

drop_fields_retweet =  [x for x in all_fields_retweet if x not in select_fields_retweet]
drop_fields_retweet

['display_text_range',
 'entities',
 'extended_entities',
 'extended_tweet',
 'favorited',
 'filter_level',
 'geo',
 'id_str',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'is_quote_status',
 'lang',
 'place',
 'possibly_sensitive',
 'quoted_status',
 'quoted_status_id',
 'quoted_status_id_str',
 'quoted_status_permalink',
 'scopes',
 'source',
 'truncated',
 'withheld_copyright',
 'withheld_in_countries']

In [36]:
tweets_dropped = tweets.withColumn('retweeted_status', tweets['retweeted_status'].dropFields(*drop_fields_retweet))

tweets_dropped.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- friends_count: long (nullable = true)
 |    |-- geo_enabled: boolean (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- id_str: string (nullable = true)
 |    |-- is_translator: boolean (nullable = true)
 |    |-- listed_count: long (nullable = true)
 |    |-- location: stri

#### Drop nested fields which contain large number of uncessary information in 'user'

In [37]:
all_fields_user = tweets.select("user.*").columns
all_fields_user

['contributors_enabled',
 'created_at',
 'default_profile',
 'default_profile_image',
 'description',
 'favourites_count',
 'followers_count',
 'friends_count',
 'geo_enabled',
 'id',
 'id_str',
 'is_translator',
 'listed_count',
 'location',
 'name',
 'profile_background_color',
 'profile_background_image_url',
 'profile_background_image_url_https',
 'profile_background_tile',
 'profile_banner_url',
 'profile_image_url',
 'profile_image_url_https',
 'profile_link_color',
 'profile_sidebar_border_color',
 'profile_sidebar_fill_color',
 'profile_text_color',
 'profile_use_background_image',
 'protected',
 'screen_name',
 'statuses_count',
 'translator_type',
 'url',
 'verified',
 'withheld_in_countries']

In [38]:
select_fields_user = ('created_at',
                     'description',
                     'favourites_count',
                     'followers_count',
                     'friends_count',
                     'geo_enabled',
                     'id',
                     'listed_count',
                     'location',
                     'name',
                     'protected',
                     'screen_name',
                     'statuses_count',
                     'verified')

drop_fields_user =  [x for x in all_fields_user if x not in select_fields_user]
drop_fields_user

['contributors_enabled',
 'default_profile',
 'default_profile_image',
 'id_str',
 'is_translator',
 'profile_background_color',
 'profile_background_image_url',
 'profile_background_image_url_https',
 'profile_background_tile',
 'profile_banner_url',
 'profile_image_url',
 'profile_image_url_https',
 'profile_link_color',
 'profile_sidebar_border_color',
 'profile_sidebar_fill_color',
 'profile_text_color',
 'profile_use_background_image',
 'translator_type',
 'url',
 'withheld_in_countries']

In [39]:
tweets_dropped = tweets.withColumn('user', tweets['user'].dropFields(*drop_fields_user))\
                        .withColumn('retweeted_status', tweets['retweeted_status'].dropFields(*drop_fields_retweet))

tweets_dropped.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- friends_count: long (nullable = true)
 |    |-- geo_enabled: boolean (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- listed_count: long (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- protected: boolean (nullable = true)
 |    |-- screen_name: string (nullable = true)
 |    |-- statuses_count: long (nullable = true)
 |    |-- verified: boolean (nullable = true)
 |-- quote_c

#### Drop nested fields which contain large number of uncessary information in 'retweeted_status.user'

In [40]:
tweets_dropped = tweets_dropped.withColumn("retweeted_status", col("retweeted_status").withField("user", col("retweeted_status.user").dropFields(*drop_fields_user)))
tweets_dropped.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- friends_count: long (nullable = true)
 |    |-- geo_enabled: boolean (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- listed_count: long (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- protected: boolean (nullable = true)
 |    |-- screen_name: string (nullable = true)
 |    |-- statuses_count: long (nullable = true)
 |    |-- verified: boolean (nullable = true)
 |-- quote_c

In [41]:
tweets_dropped.count()

99992797

In [42]:
tweets_dropped.limit(20)

coordinates,created_at,id,text,user,quote_count,reply_count,favorite_count,retweet_count,retweeted,retweeted_from,retweeted_status
null,Tue May 24 22:09:...,1529223198327287808,RT @ABC: “Why are...,{Thu Mar 05 21:39...,0,0,0,0,RT,ABC,"{null, Tue May 24..."
null,Tue May 24 22:09:...,1529223197253435392,Indiana High Scho...,{Thu May 19 00:00...,0,0,0,0,,null,null
null,Tue May 24 22:09:...,1529223198478311425,RT @jaketapper: F...,{Fri Jun 24 02:59...,0,0,0,0,RT,jaketapper,"{null, Tue May 24..."
null,Tue May 24 22:09:...,1529223198449012736,#Uvalde is just a...,{Wed Feb 26 19:03...,0,0,0,0,,null,null
null,Tue May 24 22:09:...,1529223198683942914,RT @Josh_Moon: 14...,{Wed Jul 08 02:17...,0,0,0,0,RT,Josh_Moon,"{null, Tue May 24..."
null,Tue May 24 22:09:...,1529223198675546112,RT @Jim_Jordan: L...,{Thu Feb 19 21:48...,0,0,0,0,RT,Jim_Jordan,"{null, Tue May 24..."
null,Tue May 24 22:09:...,1529223199036059648,RT @meganbang3: M...,{Mon Mar 19 02:44...,0,0,0,0,RT,meganbang3,"{null, Mon May 23..."
null,Tue May 24 22:09:...,1529223199032066050,RT @jewishaction:...,{Fri May 31 18:32...,0,0,0,0,RT,jewishaction,"{null, Tue May 24..."
null,Tue May 24 22:09:...,1529223199422316545,RT @LRiddickESPN:...,{Fri Jul 29 13:40...,0,0,0,0,RT,LRiddickESPN,"{null, Tue May 24..."
null,Tue May 24 22:09:...,1529223199174672385,@TheUSASingers My...,{Thu Sep 03 12:51...,0,0,0,0,,null,null


In [43]:
user = tweets_dropped.select('user.*')
user.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- description: string (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- geo_enabled: boolean (nullable = true)
 |-- id: long (nullable = true)
 |-- listed_count: long (nullable = true)
 |-- location: string (nullable = true)
 |-- name: string (nullable = true)
 |-- protected: boolean (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- statuses_count: long (nullable = true)
 |-- verified: boolean (nullable = true)



In [44]:
user.limit(20)

created_at,description,favourites_count,followers_count,friends_count,geo_enabled,id,listed_count,location,name,protected,screen_name,statuses_count,verified
Wed Sep 30 13:36:...,money minded💰💰 ...,31744,1909,2396,true,1311298868823371780,3,"Lagos, Nigeria",👑Chief Dozzy of ...,false,ChiffDozzy,13648,false
Thu Aug 20 12:37:...,give me the money,9324,233,1117,false,1296425927002357761,0,"enhypen, my solace.",pshlvr,false,psolonsh,4932,false
Wed Oct 21 17:33:...,🐕🐰 ArmyCaratEngene,21824,27,80,false,1318968503316066305,0,null,장미💙,false,keyxxef,7362,false
Sat Dec 05 08:09:...,Tempat Bucinin si...,5402,10,188,false,1335134070490066945,0,"Riau, Indonesia",Bby_WOOlfie🐺,false,Bby_WOOlfie,19681,false
Sun Jan 11 21:01:...,Mass communicatio...,67679,641,792,true,18879539,18,North Carolina,Dr. Vanessa Bravo...,false,VanessaBravoCR,15615,false
Tue May 15 09:19:...,Thank BTS for exp...,250380,747,682,false,996319055748390913,15,21 || she/her,ᴾʳᵒᵒᶠShani⁷ || Sa...,false,TriviaJoonShani,51919,false
Sun Jun 04 16:12:...,null,845778,845,853,false,871399252815204354,3,A BUCKEYE living ...,Terry Egan 🇺🇦,false,Terryg1979,515813,false
Thu May 08 13:12:...,High school teach...,540,192,183,false,2483734537,4,null,Liz Winfield,false,LizzyWinfield,816,false
Mon Jan 19 00:32:...,"Music, Disney, an...",101465,426,2561,true,19164128,27,Earth,Human,false,Butrphli,28366,false
Wed Jan 11 05:17:...,"Not a Cat, Is a C...",79816,32331,6330,true,819050524729286656,187,null,Public Defendering,false,fodderyfodder,47783,false


# Save the Preprocessed Results As Parquet Files

In [45]:
dir_out = 'inter_result/preprocessed'

In [ ]:
#list_blobs_pd(bucket, dir_out)

In [ ]:
#delete_folder(bucket, dir_out)

In [ ]:
#tweets_dropped.write.format("parquet").\
#    mode('overwrite').\
#    save('gs://' + bucket + '/' + dir_out)

In [46]:
blob_list = list_blobs_pd(bucket, dir_out)
blob_list.head(10)

,Name,Size
0,inter_result/preprocessed/,0
1,inter_result/preprocessed/_SUCCESS,0
2,inter_result/preprocessed/part-00000-3fc5e83a-6175-43e3-991a-856d1db34d85-c000.snappy.parquet,6714906
3,inter_result/preprocessed/part-00001-3fc5e83a-6175-43e3-991a-856d1db34d85-c000.snappy.parquet,6544339
4,inter_result/preprocessed/part-00002-3fc5e83a-6175-43e3-991a-856d1db34d85-c000.snappy.parquet,6374015
5,inter_result/preprocessed/part-00003-3fc5e83a-6175-43e3-991a-856d1db34d85-c000.snappy.parquet,6498634
6,inter_result/preprocessed/part-00004-3fc5e83a-6175-43e3-991a-856d1db34d85-c000.snappy.parquet,6419798
7,inter_result/preprocessed/part-00005-3fc5e83a-6175-43e3-991a-856d1db34d85-c000.snappy.parquet,6568402
8,inter_result/preprocessed/part-00006-3fc5e83a-6175-43e3-991a-856d1db34d85-c000.snappy.parquet,6605609
9,inter_result/preprocessed/part-00007-3fc5e83a-6175-43e3-991a-856d1db34d85-c000.snappy.parquet,6386363
